In [37]:
sc

<SparkContext master=local[*] appName=PySparkShell>

In [1]:
import json
import os

In [2]:
pdf_json = os.listdir('../document_parses/pdf_json/')
pmc_json = os.listdir('../document_parses/pmc_json/')

In [3]:
jsf = f'../document_parses/pdf_json/{pdf_json[0]}'

with open(jsf, 'r') as f:
    data = json.load(f)
    
data

{'paper_id': '0001418189999fea7f7cbe3e82703d71c85a6fe5',
 'metadata': {'title': 'Absence of surface expression of feline infectious peritonitis virus (FIPV) antigens on infected cells isolated from cats with FIP',
  'authors': [{'first': 'E',
    'middle': [],
    'last': 'Cornelissen',
    'suffix': '',
    'affiliation': {'laboratory': 'Laboratory of Virology',
     'institution': 'Ghent University',
     'location': {'addrLine': 'Salisburylaan 133',
      'postCode': '9820',
      'settlement': 'Merelbeke',
      'country': 'Belgium'}},
    'email': ''},
   {'first': 'H',
    'middle': ['L'],
    'last': 'Dewerchin',
    'suffix': '',
    'affiliation': {'laboratory': 'Laboratory of Virology',
     'institution': 'Ghent University',
     'location': {'addrLine': 'Salisburylaan 133',
      'postCode': '9820',
      'settlement': 'Merelbeke',
      'country': 'Belgium'}},
    'email': ''},
   {'first': 'E',
    'middle': ['Van'],
    'last': 'Hamme',
    'suffix': '',
    'affiliation

In [4]:
paper_id = data['paper_id']
title = data['metadata']['title']

print(paper_id)

title

0001418189999fea7f7cbe3e82703d71c85a6fe5


'Absence of surface expression of feline infectious peritonitis virus (FIPV) antigens on infected cells isolated from cats with FIP'

In [5]:
abstract = [x['text'] for x in data['abstract']]

" ".join(abstract)

'Feline infectious peritonitis virus (FIPV) positive cells are present in pyogranulomas and exudates from cats with FIP. These cells belong mainly to the monocyte/macrophage lineage. How these cells survive in immune cats is not known. In this study, FIPV positive cells were isolated from pyogranulomas and exudates of 12 naturally FIPV-infected cats and the presence of two immunologic targets, viral antigens and MHC I, on their surface was determined. The majority of the infected cells were confirmed to be cells from the monocyte/macrophage lineage. No surface expression of viral antigens was detected on FIPV positive cells. MHC I molecules were present on all the FIPV positive cells. After cultivation of the isolated infected cells, 52 AE 10% of the infected cells re-expressed viral antigens on the plasma membrane. In conclusion, it can be stated that in FIP cats, FIPV replicates in cells of the monocyte/macrophage lineage without carrying viral antigens in their plasma membrane, whic

In [6]:
body_text = [x['text'] for x in data['body_text']]

" ".join(body_text)

"Feline infectious peritonitis (FIP) is a fatal chronic disease in cats caused by a coronavirus, feline infectious peritonitis virus (FIPV), and characterized by granulomatous lesions formed at the serosae of different organs. Two forms can be distinguished. Cats suffering from the wet or effusive form have exudates in their body cavities. Exudate is absent in the second form, hence the name dry or non-effusive form. FIPV-infected cells are detected in the pyogranulomas and, based on morphology and the granulocyte/ monocyte/macrophage marker calprotectin, defined as macrophages (Weiss and Scott, 1981; Kipar et al., 1998) . Infected mononuclear cells were also isolated from exudates (Cammarata Parodi et al., 1993; Paltrinieri et al., 1999) . In one way or another, these infected cells succeed in staying alive and transmitting www.elsevier.com/locate/vetmic Veterinary Microbiology 121 (2007) [131] [132] [133] [134] [135] [136] [137] virus to new susceptible cells in the presence of a hig

### Creating a small sample dataset

In [32]:
import pandas as pd

pdfs = pd.DataFrame({'file': pdf_json})

pdf_s = pdfs.sample(100)

pdf_s

,file
11821,239a9520b2e0bfa266ff34dda9cd3fc69a55fa54.json
78260,eaa8b4bce414cb5f48f66c0f12730b0e3db0603e.json
5590,10ee080308e7ee4e7e6223c75f4666b096e469e5.json
8216,18ad65a3a27ad033c1896a292a5c8e5ba063fa7c.json
17306,33cd48a0c8515314a52b9697463093b5db96e808.json
...,...
21441,3fe504e422063819de1c2696e9e2d3ebdbab0d26.json
8878,1aa1ceba1b4b3fb638e55bf180727dabff07353c.json
24612,4955cff332e26d5a2b962a7ee278842630d1d2dd.json
12073,245265fe297924b07ccc511f97245f5636d656d7.json


In [36]:
df = pd.DataFrame(data={'paper_id':[], 'title':[], 'abstract':[], 'body_text':[]})

keys = list(df.columns)

rows = []
for pdf in pdf_s['file']:
    with open(f'../document_parses/pdf_json/{pdf}', 'r') as f:
        data = json.load(f)
    dct = {k: (data[k] if k != 'title' else data['metadata'][k]) for k in keys}
    for y in ('abstract','body_text'):
        dct[y] = " ".join([x['text'] for x in dct[y]])
        
    rows.append(dct)

df = df.append(rows)

In [39]:
df.to_pickle('n100.pkl')

In [66]:
dataset = pd.read_pickle('n100.pkl')

In [65]:
import numpy as np
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity

practice_data = dataset['body_text'][0:100]

sentences = practice_data[0].split('.')
practice_set = np.array(sentences)


model = SentenceTransformer('distilbert-base-nli-stsb-mean-tokens')
sentence_embeddings_bert = model.encode(practice_set)

g = input("Enter short description of article you are looking for: ")
query_embedded = model.encode(g)

def find_similar(vector_query, vector_total_set, k=1):
    similarity_matrix = cosine_similarity(vector_query, vector_total_set)
    np.fill_diagonal(similarity_matrix, 0)
    similarities = similarity_matrix[0]
    if k == 1:
        return [np.argmax(similarities)]
    elif k is not None:
        return np.flip(similarities.argsort()[-k:][::1])

similar_indexes = find_similar(query_embedded,sentence_embeddings_bert , 5)
print()
print("5 most similar descriptions using Sentence-Bert")
for num, index in enumerate(similar_indexes,1):
    print(str(num) + ". " + practice_set[index])


Enter short description of article you are looking for: proteins in COVID-19

5 most similar descriptions using Sentence-Bert
1.  In the case of HeLa cells (which constitutively express BST2), we did found SARS-CoV S colocalized with BST2 in the plasma membrane ( Figure 6B )
2.  Mammalian expression vectors encoding SARS-CoV M, N, S, and E were provided by G
3.  Our work is built in part on an earlier finding by another research team that the ORF7a accessory protein (encoded by SARS-CoV) inhibits the BST2 tethering of virions
4.  20, 21 Combined, these data support the assumption that enveloped virus budding and release occurring at the plasma membrane or in an intracellular compartment is subject to BST2 blocking
5.  After binding to BST2, Vpu moves BST2 toward lysosomal-and proteasomal-degradation pathways
